In [ ]:
#!pip install pandas_datareader

In [ ]:
#!pip install yfinance

In [ ]:
#!pip install html5lib

In [10]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import os
import yfinance as yf
from time import sleep
import glob

## Selenium

### Configurando WebDriver

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

### Configurando pasta de download

In [12]:
chromeOptions = webdriver.ChromeOptions()
download_path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data'
prefs = {"download.default_directory" : download_path}
chromeOptions.add_experimental_option("prefs",prefs)

In [13]:
driver = webdriver.Chrome(options=chromeOptions)

In [3]:
os.listdir()
os.chdir('data')

In [4]:
os.listdir()

['IBOVDia_21-11-22 (1).csv',
 'IBOVDia_21-11-22.csv',
 'ibov_empresas.csv',
 'ibov_setor.csv',
 'ibrx100_setor.csv',
 'ibrx_empresas.csv']

In [21]:
arquivo = !ls -1t *.csv

In [12]:
indice = 'ibov'

In [13]:
url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'

In [14]:
driver.get(url)

In [15]:
driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")

In [16]:
driver.find_element(By.LINK_TEXT, 'Download').click()

In [ ]:
driver.close()

In [41]:
os.getcwd()
files = list(glob.glob('*csv'))
print(files)

files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

print(files)

['IBOVDia_21-11-22 (1).csv', 'IBOVDia_21-11-22.csv', 'ibov_empresas.csv', 'ibov_setor.csv', 'ibrx100_setor.csv', 'ibrx_empresas.csv']
['IBOVDia_21-11-22 (1).csv', 'IBOVDia_21-11-22.csv', 'ibrx100_setor.csv', 'ibrx_empresas.csv', 'ibov_setor.csv', 'ibov_empresas.csv']


In [19]:
def get_index_stocks(index, wait=5):
    '''
        Recieves the Index name, download a file that contains the index stocks, and return the name of the downloaded file
    '''


    #Chrome WebDriver opens the index webside
    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{index.upper()}?language=pt-br'
    driver.get(url)
    driver.implicitly_wait(wait)

    driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")
    driver.implicitly_wait(wait)
    driver.find_element(By.LINK_TEXT, 'Download').click()
    driver.implicitly_wait(wait)

    #set the directory
    os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data')

    #get the files from the selectec directory and sort them by modification date
    files = files = list(glob.glob('*csv'))
    sleep(wait)
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

    return files[0]

In [25]:
index = 'ibov'

create_df(get_index_stocks(index))

C:\Users\Pedro\AppData\Local\Temp\ipykernel_13644\1526234721.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  DataFrame = pd.read_csv(file,


,setor,codigo,acao,tipo,qtde_teorica,part_%,part_%acum
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1481593024,2.840,2.840
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734632205,0.509,0.509
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327646296,0.209,2.119
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,1115681435,0.617,2.119
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,333657615,0.073,2.119
...,...,...,...,...,...,...,...
87,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,248477689,0.540,9.471
88,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1561135009,0.972,9.471
89,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON NM,255208461,0.494,9.471
90,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1100513485,1.478,9.471


## Download Index file (.csv)

In [ ]:
def get_index_stocks(index, wait=5):
    '''
        Recieves the Index name, download a file that contains the index stocks, and return the name of the downloaded file
    '''


    #Chrome WebDriver opens the index webside
    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{index.upper()}?language=pt-br'
    driver.get(url)
    driver.implicitly_wait(wait)

    driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")
    driver.implicitly_wait(wait)
    driver.find_element(By.LINK_TEXT, 'Download').click()
    driver.implicitly_wait(wait)

    #set the directory
    os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data')

    #get the files from the selectec directory and sort them by modification date
    files = files = list(glob.glob('*csv'))
    sleep(wait)
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

    return files[0]

## Get the Index DataFrame

In [24]:
def create_df(file):
    '''Essa é uma função que recebe o nome do índice da Bolsa de Valores e retorna um DataFrame com todas as empresas e seus respectivos setores'''

    #criando DataFrame
    DataFrame = pd.read_csv(file, 
                                encoding='ISO-8859-1',
                                header=1,
                                sep=';',
                                decimal=',',
                                thousands='.',
                                skipfooter=2,
                                index_col=False)
    
    #normalizando colunas
    DataFrame.columns = [re.sub('[\.()]', '', re.sub(' ', '_', unidecode(columns.lower()))) for columns in DataFrame.columns]

    return DataFrame

## Criando DataFrame com empresas da Ibovespa

In [ ]:
ibov = create_df('ibov')
ibov

In [ ]:
ibov = pd.read_csv('data/ibov_setor.csv', 
                                encoding='ISO-8859-1',
                                header=1,
                                sep=';',
                                decimal=',',
                                thousands='.',
                                skipfooter=2,
                                index_col=False)

## Criando DataFrame com empresas do IBRX100

In [ ]:
ibrx = create_df('ibrx100')

## Verificando empresas que divergem nos dois índices

In [ ]:
emp_ibrx = ibrx['codigo']
set_ibrx = set(emp_ibrx)
len(set_ibrx)

In [ ]:
emp_ibov = ibov['codigo']
set_ibov = set(emp_ibov)
len(set_ibov)

In [ ]:
print(len(set_ibov.difference(set_ibrx)))
print(set_ibov.difference(set_ibrx))

In [ ]:
print(len(set_ibrx.difference(set_ibov)))
print(set_ibrx.difference(set_ibov))

## Criando DataFrame unindo os dois índices

In [ ]:
df_empresas = pd.concat([ibrx, ibov]).drop_duplicates(subset='codigo', ignore_index=True).drop(['qtde_teorica',r'part_%',r'part_%acum'], axis=1)

## Criando lista com o código dos pricipais ativos

In [ ]:
codigo = df_empresas['codigo']

### Criando lista com o codigo das ações do Yahoo Finance

In [ ]:
codigo_yfinance = list(codigo+'.SA')
codigo_yfinance

## Yahoo Finance

In [ ]:
yf.Ticker('VALE3.SA').history('18y')